In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import random
import math

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_score

In [3]:
def NormalEquation(matX,vecY):
    X_transpose = np.transpose(matX)

    left_side = np.dot(X_transpose,matX)
    right_side = np.dot(X_transpose,vecY)

    ans = np.linalg.inv(left_side).dot(right_side)
    return ans

def CalculateMSE(x_positions, y_positions, w_list):
    x = np.array(x_positions)
    n = len(x_positions)
    MSE = 0
    for i in range(0, n):
        wx, count = 0, 0
        for w in w_list:
            wx += w * x[i, count-1] if count > 0 else w
            count += 1
        MSE += (wx - y_positions[i]) ** 2
    MSE *= 1/(n)
    return MSE

def testCalculateMSE(x_positions, y_positions, w0, w1):
    n = len(x_positions)
    MSE = 0
    for i in range(0, n):
        MSE += (w0 + w1*x_positions[i] - y_positions[i]) ** 2
    MSE *= 1/(n)
    return MSE

In [4]:
df = pd.read_csv("sin experiment/sin_noiseless_10sample.csv")
print(df["y"])

0   -1.220000e-16
1   -5.877853e-01
2   -9.510565e-01
3   -9.510565e-01
4   -5.877853e-01
5    0.000000e+00
6    5.877853e-01
7    9.510565e-01
8    9.510565e-01
9    5.877853e-01
Name: y, dtype: float64


In [5]:
k_folds = KFold(n_splits=5)

for train_index, validate_index in k_folds.split(df[["x", "x^2", "x^3"]]):
    print(train_index, validate_index)
    X_train , X_validate = df[["x", "x^2", "x^3"]].iloc[train_index],df[["x", "x^2", "x^3"]].iloc[validate_index]
    y_train , y_validate = df["y"].iloc[train_index] , df["y"].iloc[validate_index]

    new_X = [[1,1,1,1,1,1,1,1]]
    for i in np.array(X_train).transpose():
        new_X.append(list(i))
    print(new_X)
    X_matrix = np.array(new_X).transpose()
    print(X_matrix)
    print(NormalEquation(X_matrix, y_train))
    print(df[["x", "x^2", "x^3"]].shape)

[2 3 4 5 6 7 8 9] [0 1]
[[1, 1, 1, 1, 1, 1, 1, 1], [-0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8], [0.36, 0.16, 0.04, 0.0, 0.04, 0.16, 0.36, 0.64], [-0.216, -0.064, -0.008, 0.0, 0.008, 0.064, 0.216, 0.512]]
[[ 1.    -0.6    0.36  -0.216]
 [ 1.    -0.4    0.16  -0.064]
 [ 1.    -0.2    0.04  -0.008]
 [ 1.     0.     0.     0.   ]
 [ 1.     0.2    0.04   0.008]
 [ 1.     0.4    0.16   0.064]
 [ 1.     0.6    0.36   0.216]
 [ 1.     0.8    0.64   0.512]]
[-0.01076204  2.92260765  0.08968369 -3.55868716]
(10, 3)
[0 1 4 5 6 7 8 9] [2 3]
[[1, 1, 1, 1, 1, 1, 1, 1], [-1.0, -0.8, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8], [1.0, 0.64, 0.04, 0.0, 0.04, 0.16, 0.36, 0.64], [-1.0, -0.512, -0.008, 0.0, 0.008, 0.064, 0.216, 0.512]]
[[ 1.    -1.     1.    -1.   ]
 [ 1.    -0.8    0.64  -0.512]
 [ 1.    -0.2    0.04  -0.008]
 [ 1.     0.     0.     0.   ]
 [ 1.     0.2    0.04   0.008]
 [ 1.     0.4    0.16   0.064]
 [ 1.     0.6    0.36   0.216]
 [ 1.     0.8    0.64   0.512]]
[ 0.03330992  2.65935653 -0.14460659 -2

In [65]:
def old_cross_validation(X, y, folds):
    #Not done
    k_folds = KFold(n_splits=folds)

    avg_rmse = []

    for train_index, validate_index in k_folds.split(X):
        #print(train_index, validate_index)
        X_train , X_validate = X.iloc[train_index], X.iloc[validate_index]
        y_train , y_validate = y.iloc[train_index] , y.iloc[validate_index]

        new_X = [[1 for i in range(X_train.shape[0])]]
        for i in np.array(X_train).transpose():
            new_X.append(list(i))
        #print(new_X)
        X_matrix = np.array(new_X).transpose()
        model = NormalEquation(X_matrix, y_train)
        #print(X_matrix)
        #print(model)
        avg_rmse.append(CalculateMSE(np.array(X_validate), np.array(y_validate), model)**(1/2))
        #print(testCalculateMSE(np.array(X_validate), np.array(y_validate), model[0], model[1]))
    print(avg_rmse)
    return np.mean(avg_rmse)

In [ ]:
def cross_validation(X, y, folds):
    #Not done
    k_folds = KFold(n_splits=folds)

    avg_rmse = []

    for train_index, validate_index in k_folds.split(X):
        print(train_index, validate_index)
        X_train , X_validate = X.iloc[train_index], X.iloc[validate_index]
        y_train , y_validate = y.iloc[train_index] , y.iloc[validate_index]

        print(np.reshape(np.array(X_train), (1,len(X_train))))

        result = np.polyfit(np.reshape(np.array(X_train), (1,len(X_train))), np.array(y_train), 1)
        
        avg_rmse.append(CalculateMSE(X_validate, y_validate, result))

    print(avg_rmse)
    return np.mean(avg_rmse)

In [25]:
cross_validation(df[["x"]], df["y"], 5)

[1.2330359646475968, 0.8229341167821799, 0.41916844680708, 0.5765719966131401, 0.27645477344263547]


0.6656330596585264

In [66]:
print(np.array(df["x"]))
print(np.array(df[["x"]]))
cross_validation(df[["x"]], df["y"], 5)

[-1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8]
[[-1. ]
 [-0.8]
 [-0.6]
 [-0.4]
 [-0.2]
 [ 0. ]
 [ 0.2]
 [ 0.4]
 [ 0.6]
 [ 0.8]]
[2 3 4 5 6 7 8 9] [0 1]
[[-0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8]]


TypeError: expected 1D vector for x